In [7]:
import numpy as np

def log_si_probs(tags, word_sequence, log_forward, log_backward, log_forward_val, words_dict, transition, emission):
    si_probabilities = np.full((len(tags), len(word_sequence)-1, len(tags)), -np.inf)

    for i in range(len(word_sequence)-1):
        for j in range(len(tags)):
            for k in range(len(tags)):
                si_probabilities[j, i, k] = (
                    log_forward[j, i] +
                    log_backward[k, i+1] +
                    transition[j, k] +
                    emission[k, words_dict[word_sequence[i+1]]] -
                    log_forward_val
                )
    return si_probabilities

def log_si_probs_vec(num_tags, word_indices, log_forward, log_backward, log_forward_val, log_transition, log_emission):
    num_words = len(word_indices)
    si_probs = np.full((num_tags, num_words-1, num_tags), -np.inf)

    for i in range(num_words - 1):
        next_word_idx = word_indices[i+1]
        si_probs[:, i, :] = (log_forward[:, i, np.newaxis] +
                             log_backward[np.newaxis, :, i+1] +
                             log_transition +
                             log_emission[np.newaxis, :, next_word_idx] -
                             log_forward_val)
    # next_word_indices = word_indices[1:]
    # si_probs = (
    #     log_forward[:, :-1, np.newaxis] +
    #     log_backward[np.newaxis, :, 1:] +
    #     log_transition[:, :, np.newaxis] +
    #     log_emission[np.newaxis, :, next_word_indices] -
    #     log_forward_val
    # )
    return si_probs



In [8]:
import numpy as np

# Define the parameters
tags = ['Noun', 'Verb', 'Adjective']
word_sequence = ['the', 'quick', 'brown', 'fox']
word_indices = [0, 1, 2, 3]  # corresponding indices in a hypothetical vocabulary
log_forward = np.array([
    [1, 2, 3, 4],  # Values for Noun
    [5, 6, 7, 8],  # Values for Verb
    [9, 10, 11, 12]  # Values for Adjective
])
log_backward = np.array([
    [120, 110, 100, 90],  # Values for Noun
    [80, 70, 60, 50],  # Values for Verb
    [40, 30, 20, 10]  # Values for Adjective
])
log_forward_val = 10

# Define transition matrix with distinct values
transition = np.array([
    [1, 2, 3],  # From Noun to Noun, Verb, Adjective
    [4, 5, 6],  # From Verb to Noun, Verb, Adjective
    [7, 8, 9]   # From Adjective to Noun, Verb, Adjective
])
# Define emission matrix with distinct values
vocab_size = max(word_indices) + 1
emission = np.array([
    [11, 22, 33, 44],  # Emissions for Noun
    [88, 77, 66, 55],  # Emissions for Verb
    [99, 111, 222, 333]   # Emissions for Adjective
])

# Print input values
print("Tags:", tags)
print("Word Indices:", word_indices)
print("Word Sequence:", word_sequence)
print("Forward Matrix:\n", log_forward)
print("Backward Matrix:\n", log_backward)
print("Forward Value:", log_forward_val)
print("Transition Matrix:\n", transition)
print("Emission Matrix:\n", emission)

# Use the original function
si_probabilities = log_si_probs(tags, word_indices, log_forward, log_backward, log_forward_val, word_indices, transition, emission)
print("SI Probabilities from log_si_probs:\n", si_probabilities)

# Use the vectorized function
si_probabilities_vec = log_si_probs_vec(len(tags), word_indices, log_forward, log_backward, log_forward_val, transition, emission)
print("SI Probabilities from log_si_probs_vec:\n", si_probabilities_vec)

# Check if the outputs are equivalent
print("Are the outputs equivalent?", np.allclose(si_probabilities, si_probabilities_vec))


Tags: ['Noun', 'Verb', 'Adjective']
Word Indices: [0, 1, 2, 3]
Word Sequence: ['the', 'quick', 'brown', 'fox']
Forward Matrix:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
Backward Matrix:
 [[120 110 100  90]
 [ 80  70  60  50]
 [ 40  30  20  10]]
Forward Value: 10
Transition Matrix:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Emission Matrix:
 [[ 11  22  33  44]
 [ 88  77  66  55]
 [ 99 111 222 333]]
SI Probabilities from log_si_probs:
 [[[124. 140. 135.]
  [126. 120. 237.]
  [128. 100. 339.]]

 [[131. 147. 142.]
  [133. 127. 244.]
  [135. 107. 346.]]

 [[138. 154. 149.]
  [140. 134. 251.]
  [142. 114. 353.]]]
SI Probabilities from log_si_probs_vec:
 [[[124. 140. 135.]
  [126. 120. 237.]
  [128. 100. 339.]]

 [[131. 147. 142.]
  [133. 127. 244.]
  [135. 107. 346.]]

 [[138. 154. 149.]
  [140. 134. 251.]
  [142. 114. 353.]]]
Are the outputs equivalent? True
